In [0]:
!pip install transformer

ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer


In [3]:
!pip install tokenizers

     |████████████████████████████████| 3.8MB 2.8MB/s 


In [0]:
import numpy as np
import pandas as pd
import unicodedata
import string
import re
import random
import torch

In [0]:
import tokenizers

In [0]:
data = pd.read_csv("drive/My Drive/machine_tran/data1.csv")

In [0]:
from tokenizers import CharBPETokenizer

In [0]:
tokenizer = CharBPETokenizer(bert_normalizer=False)

In [35]:
tokenizer

Tokenizer(vocabulary_size=0, model=BPE, unk_token=<unk>, suffix=</w>, dropout=None, lowercase=False, unicode_normalizer=None, bert_normalizer=False, split_on_whitespace_only=False)

In [0]:
tokenizer.train(["data/eng.txt"], special_tokens=[
          "<s>",
          "<pad>",
          "</s>" ,
          "<mask>"                           
])

In [37]:
tokenizer

Tokenizer(vocabulary_size=5286, model=BPE, unk_token=<unk>, suffix=</w>, dropout=None, lowercase=False, unicode_normalizer=None, bert_normalizer=False, split_on_whitespace_only=False)

In [39]:
tokenizer.save("data/")

['data/vocab.json', 'data/merges.txt']

In [0]:
eng_data = data['en'][:2000].to_list()

In [0]:
tokenizer._tokenizer.post_processor = tokenizers.processors.BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)

In [0]:
tokenizer.enable_padding(pad_id=1, pad_token = "<pad>")

In [0]:
tokenizer.enable_truncation(max_length=1024)

In [0]:
token_data = [x.ids for x in tokenizer.encode_batch(eng_data)]

In [82]:
open("data/vocab.json").read()

'{"<s>":0,"<pad>":1,"</s>":2,"<mask>":3,"!":4,"(":5,")":6,",":7,"-":8,".":9,"0":10,"1":11,"2":12,"3":13,"4":14,"5":15,"6":16,"7":17,"8":18,":":19,";":20,"?":21,"A":22,"B":23,"C":24,"D":25,"E":26,"F":27,"G":28,"H":29,"I":30,"J":31,"K":32,"L":33,"M":34,"N":35,"O":36,"P":37,"Q":38,"R":39,"S":40,"T":41,"U":42,"V":43,"W":44,"Y":45,"Z":46,"a":47,"b":48,"c":49,"d":50,"e":51,"f":52,"g":53,"h":54,"i":55,"j":56,"k":57,"l":58,"m":59,"n":60,"o":61,"p":62,"q":63,"r":64,"s":65,"t":66,"u":67,"v":68,"w":69,"x":70,"y":71,"z":72,"·":73,"ʹ":74,"—":75,"‘":76,"’":77,"“":78,"”":79,"s</w>":80,"h</w>":81,"r</w>":82,"e</w>":83,"y</w>":84,"o</w>":85,"g</w>":86,"n</w>":87,"t</w>":88,"c</w>":89,"a</w>":90,"d</w>":91,"l</w>":92,"w</w>":93,";</w>":94,"k</w>":95,"i</w>":96,"ʹ</w>":97,"m</w>":98,"b</w>":99,"p</w>":100,"u</w>":101,":</w>":102,"”</w>":103,"f</w>":104,"5</w>":105,"3</w>":106,"4</w>":107,"6</w>":108,"z</w>":109,"x</w>":110,".</w>":111,"(</w>":112,"1</w>":113,"8</w>":114,")</w>":115,"’</w>":116,"—</w>":11

In [115]:
len(token_data[200])

98

In [0]:
token_data = torch.Tensor(token_data).long()

In [118]:
token_data.shape

torch.Size([2000, 98])

In [0]:
valid = token_data[:800]
train = token_data[800:]

In [0]:
from torch.utils.data import TensorDataset, DataLoader

In [0]:
train_data = TensorDataset(train)
valid_data = TensorDataset(valid)

In [0]:
batch_size = 100

train_loader= DataLoader(train_data,shuffle=True,batch_size=batch_size,)
valid_loader =DataLoader(valid_data,shuffle=True,batch_size=batch_size,)

In [124]:
dataiter = iter(train_loader)
sample_x= dataiter.next()[0]

sample_x.shape

torch.Size([100, 98])

In [125]:
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [0]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [0]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [0]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [0]:
class Decoder(nn.Module):

  def __init__(self, layer, N):

    super(Decoder, self).__init__()
    self.layers = clones(layer, N)
    self.norm = LayerNorm(layer.size)

  def forward(self,x, tgt_mask):

    for layer in self.layers:
      x = layer(x, tgt_mask)
    return self.norm(x)

In [0]:
class DecoderLayer(nn.Module):

  def __init__(self, size, self_attn, feed_forward, dropout):
    super(DecoderLayer, self).__init__()
    self.size = size
    self.self_attn = self_attn
    self.feed_forward = feed_forward
    self.sublayer = clones(SublayerConnection(size, dropout), 2)

  def forward(self, x, tgt_mask):

    x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
    return self.sublayer[1](x, self.feed_forward)

In [0]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

In [0]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [0]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [0]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.gelu(self.w_1(x))))

In [0]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [0]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=1024):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

In [0]:
class GP(nn.Module):

  def __init__(self, decoder, tgt_embed, generator):
    super(GP, self).__init__()
    self.decoder = decoder
    self.embed = tgt_embed
    self.generator = generator

  def forward(self, x, x_mask):

    x = self.embed(x)
    x =  self.decoder(x,x_mask)

    return x

In [0]:
def make_model(eng_vocab, N=12, d_model=768, d_ff=3072, h=12, dropout=0.1):

  c = copy.deepcopy
  attn = MultiHeadedAttention(h,d_model)
  ff   = PositionwiseFeedForward(d_model, d_ff, dropout)
  position = PositionalEncoding(d_model, dropout)


  model = GP(
      Decoder(DecoderLayer(d_model, c(attn), c(ff), dropout), N),
      nn.Sequential(Embeddings(d_model, eng_vocab), c(position)),
      Generator(d_model, eng_vocab)
  )

  for p in model.parameters():
    if p.dim() > 1:
      nn.init.xavier_uniform(p)
  return model

In [0]:
class Batch:

  def __init__(self, trg, pad=1):

    self.trg = trg[:, :-1]
    self.trg_y = trg[:, 1:]

    self.trg_mask = self.make_std_mask(self.trg,pad)

    self.ntokens = (self.trg_y !=pad).data.sum()

  @staticmethod
  def make_std_mask(tgt, pad):
    tgt_mask = (tgt != pad).unsqueeze(-2)
    tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
    return tgt_mask


In [0]:
def run_epoch(data_iter, model, loss_compute):

  start = time.time()
  total_tokens = 0
  total_loss = 0
  tokens = 0

  for i , data in enumerate(data_iter):

    trg = data[0]

    batch = Batch(trg.cuda())

    out = model.forward(batch.trg, batch.trg_mask)

    loss = loss_compute(out, batch.trg_y, batch.ntokens)

    total_loss += loss
    total_tokens += batch.ntokens
    tokens += batch.ntokens

    if i % 50 == 1:

      elapsed = time.time() - start
      print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
              (i, loss / batch.ntokens, tokens / elapsed))
      start = time.time()
      tokens = 0
  return total_loss / total_tokens

In [0]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [0]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

In [0]:
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), 
                              y.contiguous().view(-1)) / norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.item() * norm

In [0]:
def greedy_decode(model, src, max_len):
    for i in range(max_len-1):
        out = model(Variable(src), 
                    Variable(subsequent_mask(src.size(1)).type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        src = torch.cat([src, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return src

In [151]:
tokenizer.get_vocab_size()

5286

In [152]:
pad_idx = tokenizer.token_to_id("<pad>")
model = make_model(tokenizer.get_vocab_size())
model.cuda()
criterion = LabelSmoothing(size=tokenizer.get_vocab_size(),padding_idx=pad_idx,smoothing=0.1)
criterion.cuda()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


LabelSmoothing(
  (criterion): KLDivLoss()
)

In [153]:
model_opt = NoamOpt(model.embed[0].d_model, 1, 2000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

for epoch in range(70):
  model.train()
  
    
  run_epoch(train_loader,model,
           SimpleLossCompute(model.generator,criterion,model_opt))
  model.eval()
  print(run_epoch(valid_loader,model,
           SimpleLossCompute(model.generator,criterion,None)))

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Epoch Step: 1 Loss: 7.487025 Tokens per Sec: 3006.268066
Epoch Step: 1 Loss: 7.034494 Tokens per Sec: 3440.545654
tensor(7.0370, device='cuda:0')
Epoch Step: 1 Loss: 7.068389 Tokens per Sec: 3348.135986
Epoch Step: 1 Loss: 6.644729 Tokens per Sec: 3724.951660
tensor(6.6327, device='cuda:0')
Epoch Step: 1 Loss: 6.652262 Tokens per Sec: 3601.246094
Epoch Step: 1 Loss: 6.362244 Tokens per Sec: 3711.864014
tensor(6.3656, device='cuda:0')
Epoch Step: 1 Loss: 6.402306 Tokens per Sec: 3493.750732
Epoch Step: 1 Loss: 6.136723 Tokens per Sec: 3637.192627
tensor(6.1836, device='cuda:0')
Epoch Step: 1 Loss: 6.220553 Tokens per Sec: 3349.936768
Epoch Step: 1 Loss: 6.073840 Tokens per Sec: 3556.642090
tensor(6.0452, device='cuda:0')
Epoch Step: 1 Loss: 6.051891 Tokens per Sec: 3605.169189
Epoch Step: 1 Loss: 5.910289 Tokens per Sec: 3696.127197
tensor(5.9083, device='cuda:0')
Epoch Step: 1 Loss: 5.927147 Tokens per Sec: 3456.751465
Epoch Step: 1 Loss: 5.734027 Tokens per Sec: 3683.106689
tensor(5.7

In [0]:
def generate_word(words, max_len, model=model):

  token = tokenizer.encode(words).ids[:-1]
  token_word = torch.Tensor(token).long()
  tok_word = token_word[None, ...]

  model.eval()

  out = greedy_decode(model, tok_word.cuda(), max_len=max_len)

  rslt = tokenizer.decode([item.item() for item in out.data[0]])

  return rslt

In [0]:
d = next(iter(valid_loader))

In [0]:
f = d[0][0]

In [0]:
f_list = [x.item() for x in f.data]

In [0]:
test = " ".join(tokenizer.decode(f_list).split()[:])

In [187]:
test

'But on the eighth day they held a solemn assembly , because they had held the inauguration of the altar for seven days and the festival for seven days .'

In [155]:
tokenizer.encode("<s> father").ids

[0, 0, 569, 2]

In [192]:
generate_word("Jesus", 50)

'Jesus said to him : “ The children of this system of things marry and are given in marriage , under the Kingdom of God is upon you did not see . been born to me in this one who sent me in Jerusalem , ,'